In [34]:
import logging
import json
from os import system
from glob import glob
from os.path import join
from pickle import dump, load
from datetime import datetime
from collections import namedtuple


import numpy as np
import pandas as pd
import plotly.express as px
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, f1_score, confusion_matrix, precision_score , recall_score
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
import transformers
from transformers import Trainer, TrainingArguments
from transformers.integrations import TensorBoardCallback
from transformers.trainer_utils import EvaluationStrategy
from transformers.data.processors.utils import InputFeatures
from transformers.data.processors import SingleSentenceClassificationProcessor
from transformers import AutoModel, AutoConfig, AutoModelForSequenceClassification, AutoTokenizer, BertTokenizer, EarlyStoppingCallback, BatchEncoding
import optuna 

from arabert.preprocess import ArabertPreprocessor

import tensorboard_analysis 
from utilities import *

# Parameters

In [2]:
# Config
seed: int = 1

# Data 
data_proportion: float = 1.0 # propotion of data to be loaded in df
load_data: bool = True 
save_data: bool = False
test_validation_proportion: float = 0.02 # test and validation proportion from df

# Model 
model_name = "aubmindlab/bert-large-arabertv2"
from_pretrained_classifier: bool = True
pretrained_classifier_name: str = "2021-12-05-train-0.898193359375"
    
# Preprocessing 
sequence_length: int = 32
tokenize_in_batches: bool = False # Helps reduce memory footprint

# Paths
code_folder_path: str = ""

# Training 
validation_size: int = 4096
batch_size: int = 64
learning_rate: float = 1e-5
epochs: int = 6
warmup_ratio: float = 0.1
save_model_while_training: bool = True # maybe doesn't work, transformers is terrible
do_warmup: bool = False
eval_while_training: bool = True # maybe doesn't work, transformers is terrible
save_model_after_finish: bool = True # maybe doesn't work, transformers is terrible

# Etc
open_tensorboard: bool = True

## Don't touch 

In [3]:
# Config
device = torch.device("cuda")
logger = logging.getLogger(__name__)
logger.setLevel(logging.ERROR)
logging.disable(logging.WARNING)

# General
date = str(datetime.today().date())

# Data
labels = ["EGY", "GLF", "IRQ", "LEV", "NOR"]

# Preprocessing
arabert_prep = ArabertPreprocessor(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Model 
if from_pretrained_classifier:
    pretrained_classifier_path = join(code_folder_path, "models", "finalized_models", pretrained_classifier_name)
    pretrained_classifier = AutoModelForSequenceClassification.from_pretrained(pretrained_classifier_path)

# Paths
train_path= f"./models/{date}-train"
search_path = f"./models/{date}-search"
dataset_string = "{}_dataset-seqlen" + str(sequence_length)


In [4]:
assert 0 < data_proportion <= 1, "data_proportion must be right side inclusively between 0 and 1"
assert 0 <= warmup_ratio <= 1, "warmup_ratio must be inclusively between 0 and 1"
assert 0 < test_validation_proportion < 1, "test_validation_proportion must be exclusively between 0 and 1"
assert 0 < sequence_length, "sequence_length must be positive"
assert 0 < epochs, "epochs must be positive"
assert 0 < batch_size, "batch_size must be positive"
assert 0 < validation_size, "validation_size must be positive"


# Data (loading, preprocessing, tokenizing)

In [5]:
%%time
# Date to dataframe ~(2.9 s)
df = get_SMADC_folder_data(code_folder_path)
df = df.sample(frac=data_proportion)

# Encode Y ~(307 ms)
classes = df["Region"].unique()
num_labels = len(classes)
class_to_index = {class_:index for class_, index in zip(classes, range(len(classes)))}
index_to_class = {index:class_ for class_, index in zip(classes, range(len(classes)))}
df["Labels"] = df["Region"].apply(class_to_index.get)

Wall time: 3.97 s


### Further preprocessing

In [6]:
if not load_data:
    # Preprocess X ~(16min 22s)
    df["Text"] = df["Text"].apply(arabert_prep.preprocess)

    # split and ~(323ms)
    train, test = train_test_split(df, test_size=test_validation_proportion, random_state=seed)
    validate, test = train_test_split(test, test_size=len(test)-validation_size, random_state=seed)
    train.reset_index(drop=True, inplace=True)
    validate.reset_index(drop=True, inplace=True)
    test.reset_index(drop=True, inplace=True)

    # Tokenize
    if tokenize_in_batches:
        validate_encoding = batch_tokenize(tokenizer, validate["Text"], 10, sequence_length)
        test_encoding = batch_tokenize(tokenizer, test["Text"], 100, sequence_length)
        train_encoding = batch_tokenize(tokenizer, train["Text"], 500, sequence_length)
    else:
        validate_encoding = tokenize(tokenizer, validate["Text"].to_list(), sequence_length)
        test_encoding = tokenize(tokenizer, test["Text"].to_list(), sequence_length)
        train_encoding = tokenize(tokenizer, list(train["Text"]), sequence_length)

    # Make Dataset 
    validate_dataset = Dialect_dataset(validate_encoding, validate["Labels"].to_list())
    test_dataset = Dialect_dataset(test_encoding, test["Labels"].to_list())
    train_dataset = Dialect_dataset(train_encoding, train["Labels"].to_list())

# Save & load preprocessed data

In [7]:
if save_data:
    save_preprocessed_data(validate_dataset, "preprocessed_validation")
    save_preprocessed_data(test_dataset, "preprocessed_test")
    save_preprocessed_data(train_dataset, "preprocessed_train")

if load_data:
    # ~(3mins)
    validate_dataset = load_preprocessed_data("preprocessed_validation")
    test_dataset = load_preprocessed_data("preprocessed_test")
    train_dataset = load_preprocessed_data("preprocessed_train")

# Training

In [8]:
training_args = generate_training_args(
                    train_path, epochs=epochs, do_warmup=do_warmup, warmup_ratio=warmup_ratio, 
                    save_model=save_model_while_training, eval_while_training=eval_while_training, 
                    learning_rate=learning_rate,batch_size=batch_size, 
                    train_dataset_length=len(train_dataset), seed=seed
                    )

# Begin train

In [9]:
if from_pretrained_classifier:
    trainer = Trainer(
    pretrained_classifier,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=validate_dataset,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3), TensorBoardCallback()]
    )
else:
    trainer = Trainer(
        model_init=lambda:model_init(model_name, len(classes), label2id=class_to_index, id2label=index_to_class),
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=validate_dataset,
        compute_metrics=compute_metrics,
        callbacks=[EarlyStoppingCallback(early_stopping_patience=3), TensorBoardCallback()]
    )

In [10]:
if open_tensorboard:
    from tensorboard import program
    tb = program.TensorBoard()
    tb.configure(argv=[None, '--logdir', join(code_folder_path, f"models")])
    print(f"Tensorflow listening on {tb.launch()}")

Tensorflow listening on http://localhost:6006/


In [11]:
class EvaluatePleaseCallback(transformers.TrainerCallback):
    def on_save(self, args, state, control, model, **kwargs):
        trainer.evaluate()
 
trainer.add_callback(EvaluatePleaseCallback())
trainer.train()

  8%|▊         | 10000/129402 [1:15:37<14:13:36,  2.33it/s]

{'loss': 0.265, 'learning_rate': 9.227832645554166e-06, 'epoch': 0.46}


 15%|█▌        | 20000/129402 [2:27:08<13:03:07,  2.33it/s]

{'loss': 0.232, 'learning_rate': 8.455433455433457e-06, 'epoch': 0.93}


 23%|██▎       | 30000/129402 [3:38:40<11:53:12,  2.32it/s]

...

 93%|█████████▎| 120000/129402 [14:41:55<1:13:30,  2.13it/s]

{'loss': 0.1115, 'learning_rate': 7.302050972937049e-07, 'epoch': 5.56}


100%|██████████| 129402/129402 [15:50:20<00:00,  2.27it/s]


{'train_runtime': 57021.1176, 'train_samples_per_second': 145.239, 'train_steps_per_second': 2.269, 'train_loss': 0.16270215116187106, 'epoch': 6.0}


TrainOutput(global_step=129402, training_loss=0.16270215116187106, metrics={'train_runtime': 57021.1176, 'train_samples_per_second': 145.239, 'train_steps_per_second': 2.269, 'train_loss': 0.16270215116187106, 'epoch': 6.0})

# Analysis

In [35]:
evaluation = trainer.evaluate(test_dataset)
evaluation

100%|██████████| 377/377 [00:46<00:00,  8.06it/s]


{'eval_loss': 0.19884486496448517,
 'eval_macro_f1': 0.9284600252295803,
 'eval_macro_precision': 0.9328351681831611,
 'eval_macro_recall': 0.9243442769402341,
 'eval_accuracy': 0.9446705989864584,
 'eval_report': {'NOR': {'precision': 0.940982058545798,
   'recall': 0.9081795397584871,
   'f1-score': 0.9242898550724638,
   'support': 4389},
  'IRQ': {'precision': 0.9096385542168675,
   'recall': 0.9086122602482136,
   'f1-score': 0.9091251175917215,
   'support': 2659},
  'LEV': {'precision': 0.9055965559655597,
   'recall': 0.8806818181818182,
   'f1-score': 0.8929654335961189,
   'support': 3344},
  'EGY': {'precision': 0.9674431401320617,
   'recall': 0.9926602051378564,
   'f1-score': 0.9798894617063767,
   'support': 10627},
  'GLF': {'precision': 0.9405155320555189,
   'recall': 0.9315875613747954,
   'f1-score': 0.9360302581812201,
   'support': 3055},
  'accuracy': 0.9446705989864584,
  'macro avg': {'precision': 0.9328351681831611,
   'recall': 0.9243442769402341,
   'f1-scor

# Save & load

In [50]:
if save_model_after_finish:
    save_path = f'models/finalized_models/{trainer.args.output_dir.split("/")[-1]}-{model_name[model_name.rfind("/")+1:]}-{trainer.evaluate()["eval_accuracy"]}'
    trainer.save_model(save_path)
    with open(join(save_path, "evaluation.json"), "w") as file:
        file.write(json.dumps(evaluation))
        
    with open(join(save_path, "arguments.txt"), "w") as file:
        file.write(str(training_args))
    

100%|██████████| 64/64 [00:07<00:00,  8.36it/s]


# Hyperparameter search

In [ ]:
# def objective(trial: optuna.Trial):     
    
#     training_args = generate_training_args(search_path, epochs=None, do_warmup=False, save_model=False, eval_while_training=False)
#     training_args.learning_rate= trial.suggest_loguniform('learning_rate', low=4e-5, high=0.01)
#     training_args.weight_decay= trial.suggest_loguniform('weight_decay', 4e-5, 0.01)
#     training_args.num_train_epochs= trial.suggest_int('num_train_epochs', low=2, high=5)
    
#     trainer = Trainer(
#         model_init=lambda:model_init(model_name, len(classes), label2id=class_to_index, id2label=index_to_class),
#         args=training_args,
#         train_dataset=train_dataset,
#         eval_dataset=validate_dataset,
#         compute_metrics=compute_metrics,
#         callbacks=[TensorBoardCallback()]
#     )
    
#     result = trainer.train()     
    
#     return result.training_loss # Or result.training_loss["metric_name"] ps: change direction in study if necessary
   
# We want to minimize the loss! 
# study = optuna.create_study(study_name='hyper-parameter-search', direction='minimize') 
# study.optimize(func=objective, n_trials=15)

# print(study.best_value) 
# print(study.best_params) 
# print(study.best_trial)

# trainer.hyperparameter_search(n_trials=10)
